In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")  
model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws").to('cuda')

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [2]:
e_token_ids = [i for i in range(len(tokenizer)) if 'e' in tokenizer.decode([i]).lower()]

In [15]:
sentence = '''"Blake won't touch her," I said. "He's not thinking about her."'''

text =  "paraphrase: " + sentence + " </s>"

encoding = tokenizer.encode_plus(text, padding='max_length', return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")

In [16]:
outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=256,
    do_sample=True,
    top_k=120,
    top_p=0.95,
    early_stopping=True,
    num_return_sequences=5,
    num_beams=5,
    bad_words_ids=[[i] for i in e_token_ids],
    suppress_tokens=e_token_ids
)

for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print(line)

"Billy won't touch it," I said, "I'm not thinking about it."
"Billy won't touch it," I said, "It's not thinking about it."
"Billy won't touch it," I said, "I don't think about it."
"Billy won't touch it," I said, "I'm not thinking about him."
"Billy won't touch it," I said, "I'm not thinking about it".
